In [1]:
import polars as pl
import pandas as pd
import plotly
import plotly.express as px
import plotly.graph_objects as go
import json
from geopy.distance import great_circle
import openrouteservice
from openrouteservice import convert
import sys
sys.path.append('..')
import keys


In [78]:
df_data = pl.read_json(r'data.json')
df2 = pd.read_json(r'data.json')
df_old_data = pl.read_excel(r'Lojas Assaí.xlsx')

In [79]:
df_inaug22 = pl.read_csv('T2022.csv')
df_inaug23 = pl.read_csv('T2023.csv')
df_inaug24 = pl.read_csv('T2024.csv')
df_inaug_data = pl.concat([
    df_inaug24.rename({"column_0": "value"}),
    df_inaug23.rename({"column_0": "value"}),
    df_inaug22.rename({"column_0": "value"})
])


In [80]:
def start_pipeline(dataf):
    return dataf.clone()

def drop_columns_new(dataf):
    dataf = dataf.drop('url', 'subRegiao', 'subRegiaoTid', 'telefone', 'televendas', 'horario', 'email', 'ico_sust', 'voce_encontra', 'destaques', 'eslug', 'eid', 'e', 'tid', 'whatsapp', 'mapa', 'complemento', 'nid', 'cep', 'loja_id')
    return dataf

def drop_columns_old(dataf):
    dataf = dataf.drop('Unnamed: 4', 'Código Município')
    return dataf

def strip_chars_new(dataf):
    dataf = dataf.with_columns(pl.col("n").str.strip_chars())
    return dataf

def strip_chars_old(dataf):
    dataf = dataf.with_columns(pl.col("Unidade").str.strip_chars())
    return dataf

def rename_columns(dataf):
    dataf = dataf.rename({'n':'Unidade', 'c':'Município', 'uf':'UF', 'lat':'LAT', 'lon':'LONG', 'logradouro':'Endereço'})
    dataf = dataf.cast({"LAT": pl.Float64, "LONG": pl.Float64})
    return dataf

def sort_columns(dataf):
    dataf = dataf.select(['Unidade','Endereço','Município','UF','LAT','LONG'])
    return dataf

def format_names(dataf):
    dataf = dataf.with_columns(
        pl.col("value")
        .str.strip_chars()  # Remove trailing spaces
        .str.replace(r" - .*", "", literal=False)  # Remove anything after the "-"
        .str.to_lowercase()  # Convert to lowercase
    )
    return dataf

def filter_tri(dataf):
    dataf = dataf.filter(~pl.col("column_1").is_in(["1T2022", "2T2022"]))
    return dataf




In [81]:
df = (df_data
 .pipe(start_pipeline)
 .pipe(drop_columns_new)
 .pipe(strip_chars_new)
 .pipe(rename_columns)
 .pipe(sort_columns))

df_old = (df_old_data
 .pipe(start_pipeline)
 .pipe(strip_chars_old)
 .pipe(drop_columns_old))

df_inaug = (df_inaug_data
 .pipe(start_pipeline)
 .pipe(format_names)
 .pipe(filter_tri))

df_inaug

value,column_1
str,str
"""assaí vitória aeroporto""","""2T2024"""
"""assaí marginal tietê vila mari…","""1T2024"""
"""assaí santa rosa""","""1T2024"""
"""assaí zona norte""","""1T2024"""
"""assaí cidade tiradentes""","""1T2024"""
…,…
"""assaí guaianases""","""3T2022"""
"""assaí araraquara""","""3T2022"""
"""assaí cabula""","""3T2022"""


In [82]:
# Lowercase columns
df_lower = df.clone()
df_old_lower = df_old.clone()

# Check for intersection etc
only_in_df1 = df.filter(~pl.col("Unidade").str.to_lowercase().is_in(df_old['Unidade'].str.to_lowercase())).with_columns(pl.lit("Nova").alias("status"))
only_in_df2 = df_old.filter(~pl.col('Unidade').str.to_lowercase().is_in(df["Unidade"].str.to_lowercase())).with_columns(pl.lit("Fechou").alias("status"))
common_in_both = df_old.filter(pl.col("Unidade").str.to_lowercase().is_in(df['Unidade'].str.to_lowercase())).with_columns(pl.lit("Antigas").alias("status"))

In [83]:
# Combine the results 
df_diff = pl.concat([
    only_in_df1.rename({"Unidade": "value"}),
    only_in_df2.rename({"Unidade": "value"}),
    common_in_both.rename({"Unidade": "value"})
])

In [84]:
df_diff = df_diff.with_columns(
    pl.col("value").str.to_lowercase().alias("value_lower")
)

# Ensure compatibility
df_inaug = df_inaug.with_columns(pl.col("value").cast(pl.Utf8))

# Perform a left join to add `column_1` from df_inaug to df_diff based on the temporary lowercase column
df_diff = df_diff.join(df_inaug, left_on="value_lower", right_on="value", how="left")

In [ ]:
df_diff = df_diff.with_columns(
    pl.when(pl.col("column_1").is_not_null())  # Check if 'column_1' exists after join
    .then(pl.lit("Nova"))  # Update status to 'Nova'
    .otherwise(pl.col("status"))  # Keep the original status otherwise
    .alias("status")
)

# Remove the temporary lowercase column after the join
df_diff = df_diff.drop("value_lower")

In [89]:
# df.write_excel(r"lojasAssai.xlsx", worksheet='lojasSite')
df_diff.write_excel(r"lojasAssaiDiff_notcorrect.xlsx", worksheet='lojasSite')


---
Mudar os valores de status para os corretos
---



In [2]:
df_diff = pl.read_excel(r"lojasAssaiDiff_corrected.xlsx")
df_diff

value,Endereço,Município,UF,LAT,LONG,status,column_1
str,str,str,str,f64,f64,str,str
"""Assaí Acrissul""","""Avenida Fábio Zahran, 7.919 Ja…","""Campo Grande""","""MS""",-20.48716,-54.622101,"""Antigas""",null
"""Assaí Adélia Franco""","""Avenida Adélia Franco 3735 - A…","""Aracaju""","""SE""",-10.948019,-37.071057,"""Conversao""","""4T2022"""
"""Assaí Aeroporto Congonhas""","""Av Washington Luís, nº 5859, S…","""São Paulo""","""SP""",-23.630161,-46.668179,"""Nova""","""4T2022"""
"""Assaí Águia de Haia""","""Avenida Águia de Haia, 2.636 P…","""São Paulo""","""SP""",-23.538604,-46.479809,"""Antigas""",null
"""Assaí Alcântara""","""Rua Doutor Alfredo Backer, 605…","""São Gonçalo""","""RJ""",-22.819909,-43.003935,"""Antigas""",null
…,…,…,…,…,…,…,…
"""Assaí Vila Sônia""","""Avenida Professor Francisco Mo…","""São Paulo""","""SP""",-23.591995,-46.731717,"""Antigas""",null
"""Assaí Vitória Aeroporto""","""Av. Fernando Ferrari, 2870""","""Vitória""","""ES""",-20.255622,-40.2894,"""Nova""","""2T2024"""
"""Assaí Vitória da Conquista""","""Avenida Anel de Contorno s/n F…","""Vitória da Conquista""","""BA""",-14.849299,-40.884204,"""Antigas""",null


In [3]:
# Define the custom sort order
status_order = ["Nova", "Conversao", "Organica", "Ex Combo Extra", "Antigas", "Fechou"]

# Add a temporary column for sorting based on status_order using conditional expressions
df_diff = df_diff.with_columns(
    pl.when(pl.col("status") == "Nova").then(0)
    .when(pl.col("status") == "Conversao").then(1)
    .when(pl.col("status") == "Organica").then(2)
    .when(pl.col("status") == "Ex Combo Extra").then(3)
    .when(pl.col("status") == "Antigas").then(4)
    .when(pl.col("status") == "Fechou").then(5)
    .otherwise(-1)  # Assign -1 or any default value for unexpected statuses
    .alias("status_order")
)

# Sort the DataFrame by the temporary 'status_order' column
df_diff = df_diff.sort("status_order")

# Drop the temporary sorting column as it's no longer needed
df_diff = df_diff.drop("status_order")

# Separate the DataFrame into active and inactive stores
active_statuses = ["Nova", "Conversao", "Organica", "Ex Combo Extra"]
inactive_statuses = ["Antigas", "Fechou"]

active_df = df_diff.filter(pl.col("status").is_in(active_statuses))
inactive_df = df_diff.filter(pl.col("status").is_in(inactive_statuses))

# Proceed with distance calculations only on active_df

In [4]:
import time

# Function to calculate route with retries
def calculate_route(client, coords, max_retries=5):
    for attempt in range(max_retries):
        try:
            route = client.directions(
                [coords[0], coords[1]],
                profile='driving-car',
                format='geojson'
            )
            return route['features'][0]['geometry']['coordinates']
        except openrouteservice.exceptions.ApiError as e:
            if "Rate limit exceeded" in str(e):
                wait_time = 2 ** attempt
                print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"API error: {e}")
                break
        except Exception as e:
            print(f"Unexpected error: {e}")
            break
    return None

# Initialize the OpenRouteService client
client = openrouteservice.Client(key=keys.API_OPENROUTESERVICE)

# Create lists for the new columns, initialized with None for all rows
closest_values_old = [None] * active_df.height
closest_distances_old = [None] * active_df.height
closest_values_old_LAT = [None] * active_df.height
closest_values_old_LONG = [None] * active_df.height
closest_values_all = [None] * active_df.height
closest_distances_all = [None] * active_df.height
route_geometries_old = [None] * active_df.height
route_geometries_all = [None] * active_df.height

# Iterate through each active row for distance calculations
for index, row in enumerate(active_df.iter_rows(named=True)):
    current_coords = (row["LAT"], row["LONG"])
    
    # Calculate distances to "Antigas" values
    distances_old = [
        (other_row["value"], great_circle(current_coords, (other_row["LAT"], other_row["LONG"])).kilometers)
        for other_row in inactive_df.iter_rows(named=True)
    ]
    
    # Find the closest "Antigas" value
    if distances_old:
        closest_value_old, min_distance_old = min(distances_old, key=lambda x: x[1])
        closest_coords_old = next(row for row in inactive_df.iter_rows(named=True) if row["value"] == closest_value_old)
        closest_values_old[index] = closest_value_old
        closest_distances_old[index] = min_distance_old
        closest_values_old_LAT[index] = closest_coords_old["LAT"]
        closest_values_old_LONG[index] = closest_coords_old["LONG"]
    
    # Calculate distances to all other active values
    distances_all = [
        (other_row["value"], great_circle(current_coords, (other_row["LAT"], other_row["LONG"])).kilometers)
        for other_row in active_df.iter_rows(named=True)
        if other_row["value"] != row["value"]  # Skip itself
    ]
    
    # Find the closest value among all active stores
    if distances_all:
        closest_value_all, min_distance_all = min(distances_all, key=lambda x: x[1])
        closest_values_all[index] = closest_value_all
        closest_distances_all[index] = min_distance_all
    
    # Calculate route for closest "Antigas" store with retries
    if closest_values_old[index]:
        route_old = calculate_route(
            client, 
            [(row["LONG"], row["LAT"]), (closest_values_old_LONG[index], closest_values_old_LAT[index])]
        )
        route_geometries_old[index] = route_old
    
    # Calculate route for closest store among all active with retries
    if closest_values_all[index]:
        closest_coords_all = active_df.filter(pl.col("value") == closest_values_all[index]).select(["LONG", "LAT"]).to_dicts()[0]
        route_all = calculate_route(
            client, 
            [(row["LONG"], row["LAT"]), (closest_coords_all["LONG"], closest_coords_all["LAT"])]
        )
        route_geometries_all[index] = route_all

c:\Users\VHENEQUIM\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\VHENEQUIM\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\VHENEQUIM\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Users\VHENEQUIM\AppData\Local\Programs\Python\Python312\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceede

In [5]:
# Add the new columns to active_df
active_df = active_df.with_columns([
    pl.Series("closest_value_old", closest_values_old),
    pl.Series("closest_distance_old", closest_distances_old),
    pl.Series("closest_value_old_LAT", closest_values_old_LAT),
    pl.Series("closest_value_old_LONG", closest_values_old_LONG),
    pl.Series("closest_value_all", closest_values_all),
    pl.Series("closest_distance_all", closest_distances_all),
    pl.Series("route_geometry_old", route_geometries_old),
    pl.Series("route_geometry_all", route_geometries_all)
])

# List of new columns added to active_df
new_columns = [
    "closest_value_old",
    "closest_distance_old",
    "closest_value_old_LAT",
    "closest_value_old_LONG",
    "closest_value_all",
    "closest_distance_all",
    "route_geometry_old",
    "route_geometry_all"
]

# Add missing columns to inactive_df with null values
inactive_df = inactive_df.with_columns([
    pl.lit(None).alias("closest_value_old"),
    pl.lit(None).alias("closest_distance_old"),
    pl.lit(None).alias("closest_value_old_LAT"),
    pl.lit(None).alias("closest_value_old_LONG"),
    pl.lit(None).alias("closest_value_all"),
    pl.lit(None).alias("closest_distance_all"),
    pl.lit(None).alias("route_geometry_old"),
    pl.lit(None).alias("route_geometry_all")
])

# Combine active_df and inactive_df back into df_diff
df_diff = pl.concat([active_df, inactive_df], rechunk=True)

In [9]:
# Define color_map if not already defined
# Example:
color_map = {
    "Nova": "#4CAF50",      # Softer green
    "Conversao": "#64B5F6",  # Light blue
    "Organica": "#FFB74D",   # Muted orange
    "Ex Combo Extra": "#9575CD",  # Soft purple
    "Antigas": "#BDBDBD",    # Light grey
    "Fechou": "#EF9A9A"      # Light red
}

df_pandas = df_diff
# Add a dummy column for size with all values set to 1
df_pandas = df_pandas.with_columns(pl.lit(1).alias("dummy_column_for_size"))

# Ensure color_map is defined
if 'color_map' not in locals():
    color_map = {
        "Nova": "green",
        "Conversao": "blue",
        "Organica": "orange",
        "Ex Combo Extra": "purple",
        "Antigas": "red",
        "Fechou": "grey"
    }

# Create the scatter mapbox figure
fig = px.scatter_mapbox(
    df_pandas, 
    lat="LAT", 
    lon="LONG", 
    hover_name="value",
    hover_data={'status': True, 'LAT': True, 'LONG': True},
    zoom=4,
    height=600,
    color='status',
    color_discrete_map=color_map,
    size='dummy_column_for_size',
    size_max=10,
)

# Update hover information
fig.update_traces(
    hovertemplate=(
        '%{hovertext}<br>'
        'Status: %{marker.color}<br>'
        'Latitude: %{lat}<br>'
        'Longitude: %{lon}<br>'
    )
)

# Add lines for closest "Antigas" stores
lines = []
for idx, row in enumerate(df_diff.iter_rows(named=True)):
    if (row['status'] in active_statuses and 
        row['closest_value_old_LAT'] is not None and 
        row['closest_value_old_LONG'] is not None):
        lines.append(go.Scattermapbox(
            mode="lines",
            lon=[row['LONG'], row['closest_value_old_LONG']],
            lat=[row['LAT'], row['closest_value_old_LAT']],
            marker={'size': 1},
            line=dict(width=2, color=color_map.get(row['status'], 'grey')),
            hoverinfo='none',
            showlegend=False
        ))

# Add all lines to the figure at once
if lines:
    fig.add_traces(lines)

# Add route lines
route_lines = []
for row in df_diff.iter_rows(named=True):
    if row['route_geometry_old']:
        route_coords = row['route_geometry_old']
        try:
            lons, lats = zip(*route_coords)
            route_lines.append(go.Scattermapbox(
                mode="lines",
                lon=lons,
                lat=lats,
                line=dict(width=2, color=color_map.get(row['status'], 'grey')),
                hoverinfo='none',
                showlegend=False
            ))
        except Exception as e:
            print(f"Error processing route coordinates: {e}")

# Add all route lines to the figure
if route_lines:
    fig.add_traces(route_lines)

# Update layout
fig.update_layout(
    mapbox_style="carto-positron",
    autosize=True,
    height=1080,
    margin={"r":0,"t":0,"l":0,"b":0}
)

# Convert the figure to JSON for HTML embedding
plot_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)

# Prepare the data for search functionality (only active stores)
search_data = active_df.select(['LAT', 'LONG', 'value', 'status']).to_dicts()
search_data_json = json.dumps(search_data)

# Create HTML content with embedded Plotly figure and search functionality
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        #search-container {{
            position: absolute;
            top: 10px;
            left: 10px;
            z-index: 1000;
            background-color: white;
            padding: 10px;
            border-radius: 5px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2);
        }}
        #search-input {{
            padding: 5px;
            width: 200px;
            margin-bottom: 5px;
        }}
        #search-results {{
            max-height: 200px;
            overflow-y: auto;
            border: 1px solid #ccc;
            display: none;
            background-color: white;
        }}
        .result-item {{
            padding: 5px;
            cursor: pointer;
        }}
        .result-item:hover {{
            background-color: #f0f0f0;
        }}
    </style>
</head>
<body>
    <div id="search-container">
        <input type="text" id="search-input" placeholder="Search for a store...">
        <div id="search-results"></div>
    </div>
    <div id="mapDiv"></div>

    <script>
        var plotlyData = {plot_json};
        var searchData = {search_data_json};
        
        Plotly.newPlot('mapDiv', plotlyData.data, plotlyData.layout);

        var searchInput = document.getElementById('search-input');
        var searchResults = document.getElementById('search-results');

        searchInput.addEventListener('input', function() {{
            var searchTerm = this.value.toLowerCase();
            
            var results = searchData.filter(function(point) {{
                return point.value.toLowerCase().includes(searchTerm);
            }});
            
            displayResults(results);
        }});

        function displayResults(results) {{
            searchResults.innerHTML = '';
            if (results.length > 0) {{
                results.forEach(function(result) {{
                    var div = document.createElement('div');
                    div.className = 'result-item';
                    div.textContent = result.value + ' (' + result.status + ')';
                    div.onclick = function() {{ 
                        centerOnPoint(result.LAT, result.LONG);
                        searchInput.value = result.value;
                        hideSearchResults();
                    }};
                    searchResults.appendChild(div);
                }});
                searchResults.style.display = 'block';
            }} else {{
                hideSearchResults();
            }}
        }}

        function centerOnPoint(lat, lon) {{
            Plotly.relayout('mapDiv', {{
                'mapbox.center': {{ lat: lat, lon: lon }},
                'mapbox.zoom': 15
            }});
        }}

        function hideSearchResults() {{
            searchResults.style.display = 'none';
        }}

        // Hide search results when clicking outside
        document.addEventListener('click', function(event) {{
            var searchContainer = document.getElementById('search-container');
            if (!searchContainer.contains(event.target)) {{
                hideSearchResults();
            }}
        }});

        // Hide search results when pressing Esc key
        document.addEventListener('keydown', function(event) {{
            if (event.key === 'Escape') {{
                hideSearchResults();
            }}
        }});

        // Prevent hiding when clicking inside the search container
        var searchContainer = document.getElementById('search-container');
        searchContainer.addEventListener('click', function(event) {{
            event.stopPropagation();
        }});
    </script>
</body>
</html>
"""

# Write the HTML content to a file
with open("localizacoesAssai_with_polars_search.html", "w") as f:
    f.write(html_content)

print("Map with Polars-compatible search functionality has been saved as 'localizacoesAssai_with_polars_search.html'")
print("Please open this file in a web browser and check the browser's console for debugging information.")

Map with Polars-compatible search functionality has been saved as 'localizacoesAssai_with_polars_search.html'
Please open this file in a web browser and check the browser's console for debugging information.
